In [18]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns

#file = 'Users/admin/Desktop/Springboard/FPA_FOD_20170508.sqlite'
con = sqlite3.connect('FPA_FOD_20170508.sqlite')
cursor = con.cursor()
print('Opened Database Successfully')

Opened Database Successfully


In [19]:
tup_names = []
table_names = []
for row in cursor.execute("SELECT name FROM sqlite_master WHERE type='table';"):
    tup_names.append(row)

for name in tup_names:
    table_names.append(name[0])

print(table_names)

['spatial_ref_sys', 'spatialite_history', 'sqlite_sequence', 'geometry_columns', 'spatial_ref_sys_aux', 'views_geometry_columns', 'virts_geometry_columns', 'geometry_columns_statistics', 'views_geometry_columns_statistics', 'virts_geometry_columns_statistics', 'geometry_columns_field_infos', 'views_geometry_columns_field_infos', 'virts_geometry_columns_field_infos', 'geometry_columns_time', 'geometry_columns_auth', 'views_geometry_columns_auth', 'virts_geometry_columns_auth', 'sql_statements_log', 'SpatialIndex', 'ElementaryGeometries', 'KNN', 'Fires', 'idx_Fires_Shape', 'idx_Fires_Shape_node', 'idx_Fires_Shape_rowid', 'idx_Fires_Shape_parent', 'NWCG_UnitIDActive_20170109']


In [29]:
fire_table = pd.read_sql_query("SELECT * FROM Fires WHERE FIRE_SIZE_CLASS >= 'G' AND FIRE_YEAR >= 2014", con)

In [30]:
print(fire_table.info())
print(fire_table.columns)
#print(fire_table.describe)
print(fire_table.shape)
#print(fire_table.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 395 entries, 0 to 394
Data columns (total 39 columns):
OBJECTID                      395 non-null int64
FOD_ID                        395 non-null int64
FPA_ID                        395 non-null object
SOURCE_SYSTEM_TYPE            395 non-null object
SOURCE_SYSTEM                 395 non-null object
NWCG_REPORTING_AGENCY         395 non-null object
NWCG_REPORTING_UNIT_ID        395 non-null object
NWCG_REPORTING_UNIT_NAME      395 non-null object
SOURCE_REPORTING_UNIT         395 non-null object
SOURCE_REPORTING_UNIT_NAME    395 non-null object
LOCAL_FIRE_REPORT_ID          142 non-null object
LOCAL_INCIDENT_ID             139 non-null object
FIRE_CODE                     349 non-null object
FIRE_NAME                     394 non-null object
ICS_209_INCIDENT_NUMBER       381 non-null object
ICS_209_NAME                  381 non-null object
MTBS_ID                       355 non-null object
MTBS_FIRE_NAME                355 non-null obje

In [31]:
#Select relevant columns
fire_table_focused = fire_table[['FOD_ID','SOURCE_SYSTEM_TYPE','SOURCE_SYSTEM','FIRE_YEAR','DISCOVERY_DATE','DISCOVERY_DOY','STAT_CAUSE_DESCR','CONT_DATE','CONT_DOY','FIRE_SIZE','FIRE_SIZE_CLASS','LATITUDE','LONGITUDE','OWNER_DESCR','STATE','COUNTY']]

#Select STATE and unique identifier column
by_state = fire_table_focused[['FOD_ID','STATE']]

#Count fires by state to determine which State will have the most data to build a model from 
by_state_group = by_state.groupby(by='STATE').count().rename(columns={'FOD_ID':'Total Fires'})
by_state_group = by_state_group.sort_values(by=['Total Fires'],ascending=False)

print(by_state_group)
print(by_state_group.sum())

#CA has had 8455 fires since the year 2000 and has available historical weather data 



       Total Fires
STATE             
AK             143
OR              47
CA              44
ID              36
WA              35
MT              16
AZ              16
TX              12
OK              11
FL              10
NM               6
NV               6
SD               3
WY               2
LA               2
KS               2
CO               2
UT               1
NE               1
Total Fires    395
dtype: int64


In [32]:
#Select only data from CA
ca_fires = fire_table_focused[fire_table_focused['STATE'] == 'CA']

by_county_group = ca_fires.groupby('COUNTY').count()['FOD_ID'].sort_values(ascending=False)

print(by_county_group)

#Counties are labeled inconsistently

COUNTY
105         8
Siskiyou    6
015         4
Shasta      3
023         3
Trinity     1
017         1
019         1
065         1
071         1
073         1
107         1
Sonoma      1
Inyo        1
Lake        1
Lassen      1
Madera      1
Modoc       1
Napa        1
003         1
Name: FOD_ID, dtype: int64


In [33]:
#GOOGLE MAPS API for reverse geocoding missing counties
import googlemaps
from datetime import datetime
import numpy as np

gmaps = googlemaps.Client(key='API_KEY')


#Create zipped list of coordinate tuples
lat_list = []
lon_list = []

for row in ca_fires['LATITUDE']:
    lat_list.append(row)

for row in ca_fires['LONGITUDE']:
    lon_list.append(row)
    
coords = list(zip(lat_list,lon_list))


In [34]:
#Reverse geocode coordinates to retrieve county names
county_names = []

for row in coords:
    county = gmaps.reverse_geocode(row,result_type='administrative_area_level_2')[0]['address_components'][0]['long_name']
    county_names.append(county)


In [40]:
#Check that geocoded counties match existing county names
ca_fires.loc[:,'GEO_COUNTY'] = county_names
print(ca_fires[['COUNTY','GEO_COUNTY']])

       COUNTY             GEO_COUNTY
1    Siskiyou        Siskiyou County
6    Siskiyou        Siskiyou County
7    Siskiyou        Siskiyou County
8      Shasta          Shasta County
17   Siskiyou        Siskiyou County
18   Siskiyou        Siskiyou County
19   Siskiyou        Siskiyou County
20     Shasta          Shasta County
21    Trinity         Trinity County
22     Madera          Madera County
42     Shasta          Shasta County
43      Modoc           Modoc County
94        073       San Diego County
95        065       Riverside County
99        017       El Dorado County
115       015       Del Norte County
116       015       Del Norte County
117       015       Del Norte County
118       023        Humboldt County
119       107          Tulare County
132       105         Trinity County
133       105         Trinity County
134       105         Trinity County
135       105         Trinity County
136       105         Trinity County
137       105         Trinity County
1

In [37]:
#Determine which counties have had the most fires
by_county = ca_fires.groupby(by='GEO_COUNTY').count()['FOD_ID'].sort_values(ascending=False)
print(by_county)

#Check that number of counties does not exceed total counties in CA (58)
print(by_county.index.nunique())

GEO_COUNTY
Trinity County           10
Siskiyou County           6
Del Norte County          4
Shasta County             3
Lake County               2
Napa County               2
Humboldt County           2
Madera County             2
Tulare County             1
Amador County             1
El Dorado County          1
Fresno County             1
Inyo County               1
Mendocino County          1
Lassen County             1
Modoc County              1
Riverside County          1
San Bernardino County     1
San Diego County          1
Tehama County             1
Alpine County             1
Name: FOD_ID, dtype: int64
21
